# Local Search Algorithm for TimeTabling Problems

## Importing Libraries

In [19]:
from typing import List, Dict
import random
import json
import random
from copy import deepcopy

## Evaluate Solution

In [20]:
def evaluate_solution(schedule: List[Dict]) -> int:
    conflicts = 0
    used_times = {}
    used_rooms = {}
    used_teachers = {}

    for class_info in schedule:
        for day_schedule in class_info['schedule']:
            for period in day_schedule['periods']:
                day = day_schedule['day']
                time = period.get('time')
                teacher = period.get('teacher')
                room = period.get('room')

                time_key = (day, time)
                room_key = (day, time, room)
                teacher_key = (day, time, teacher)

                # Conflito de horários
                if time_key in used_times:
                    conflicts += 1
                used_times[time_key] = True

                # Conflito de salas
                if room_key in used_rooms:
                    conflicts += 1
                used_rooms[room_key] = True

                # Conflito de professores
                if teacher_key in used_teachers:
                    conflicts += 1
                used_teachers[teacher_key] = True

                print(conflicts)

    return conflicts

## Generate Initial Solution

In [21]:
def generate_initial_solution(
    classes: List[Dict],
    teachers: List[Dict],
    rooms: List[Dict],
    times: List[Dict],
    subjects: List[Dict],
    days: List[Dict]
) -> List[Dict]:

    schedule = []
    for class_info in classes:
        class_schedule = []

        for day in days:
            day_name = day['name']
            day_schedule = {'day': day_name, 'periods': []}

            for time in times:
                time_info = f"{time['start']} - {time['end']}"
                available_rooms = [room for room in rooms if day['id'] in room['days'] and time['id'] in room['times']]

                suitable_rooms = [room for room in available_rooms if room['capacity'] >= class_info['studentsAmount']]
                random_room = random.choice(suitable_rooms) if suitable_rooms else None

                if random_room:
                    room_name = random_room['name']
                else:
                    room_name = "No suitable room available"

                random_teacher = random.choice(teachers)
                subject = random.choice(random_teacher['subjects'])
                teacher_name = random_teacher['name']

                period = {
                    "day": day_name,
                    "time": time_info,
                    "subject": next((sub['name'] for sub in subjects if sub['id'] == subject), None),
                    "teacher": teacher_name,
                    "room": room_name
                }
                day_schedule['periods'].append(period)

            class_schedule.append(day_schedule)

        schedule.append({"id": class_info['id'], "name": class_info['name'], "schedule": class_schedule})

    return schedule

## Metrics Evalutator

In [22]:
import sys 
sys.path.insert(1, '../')

from models_evaluator import MetricsEvaluator

metrics_evaluator = MetricsEvaluator("Local Search Algorithm", "conflicts")

Creating MetricsEvaluator of Model: Local Search Algorithm, starting evaluation


In [23]:

def generate_initial_solution(
    classes: List[Dict], 
    teachers: List[Dict], 
    rooms: List[Dict], 
    times: List[Dict],
    subjects: List[Dict],
    days: List[Dict]
) -> List[Dict]:

    schedule = []
    for class_info in classes:
        class_schedule = []

        for day in days:
            day_name = day['name']
            day_schedule = {'day': day_name, 'periods': []}

            for time in times:
                time_info = f"{time['start']} - {time['end']}"
                available_rooms = [room for room in rooms if day['id'] in room['days'] and time['id'] in room['times']]

                suitable_rooms = [room for room in available_rooms if room['capacity'] >= class_info['studentsAmount']]
                random_room = random.choice(suitable_rooms) if suitable_rooms else None

                if random_room:
                    room_name = random_room['name']
                else:
                    room_name = "No suitable room available"

                random_teacher = random.choice(teachers)
                subject = random.choice(random_teacher['subjects'])
                teacher_name = random_teacher['name']

                period = {
                    "day": day_name,
                    "time": time_info,
                    "subject": next((sub['name'] for sub in subjects if sub['id'] == subject), None),
                    "teacher": teacher_name,
                    "room": room_name
                }
                day_schedule['periods'].append(period)

            class_schedule.append(day_schedule)

        schedule.append({"id": class_info['id'], "name": class_info['name'], "schedule": class_schedule})

    return schedule

In [24]:
def get_available_rooms(day_id: int, time_id: int, rooms) -> List[Dict]:
        return [room for room in rooms if day_id in room['days'] and time_id in room['times']]

In [25]:


def local_search(
    classes: List[Dict],
    teachers: List[Dict],
    rooms: List[Dict],
    times: List[Dict],
    subjects: List[Dict],
    days: List[Dict],
    max_iterations: int = 1000
) -> List[Dict]:

        best_solution = generate_initial_solution(classes, teachers, rooms, times, subjects, days)
        best_score = evaluate_solution(best_solution)

        for _ in range(max_iterations):
            metrics_evaluator.start_iteration()
            candidate_solution = deepcopy(best_solution)

            for class_info in candidate_solution:
                for day_schedule in class_info['schedule']:
                    day_id = next(day['id'] for day in days if day['name'] == day_schedule['day'])
                    for idx, period in enumerate(day_schedule['periods']):
                        available_rooms = get_available_rooms(day_id, idx + 1, rooms)
                        if available_rooms:
                            random_room = random.choice(available_rooms)
                            period['room'] = random_room['name']
                        else:
                            period['room'] = "No room available"

            candidate_score = evaluate_solution(candidate_solution)
        
            if candidate_score < best_score:
                best_solution = deepcopy(candidate_solution)
                best_score = candidate_score

            metrics_evaluator.finish_iteration(_, candidate_solution, candidate_score)

        metrics_evaluator.finish_evaluation()
        return metrics_evaluator

In [26]:
if __name__ == "__main__":
    with open('../implementations/data/inputs/input1.json', 'r') as file:
        data = json.load(file)

    classes = data['classes']
    teachers = data['teachers']
    rooms = data['rooms']
    times = data['times']
    subjects = data['subjects']
    days = data['days']

    metrics_evaluator = local_search(classes, teachers, rooms, times, subjects, days)

    # with open('../implementations/data/outputs/timetabling_output.json', 'w') as file:
    #     json.dump(best_schedule, file, indent=2, ensure_ascii=False)


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
4
6
8
9
11
13
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
51
54
55
57
58
60
62
64
66
69
71
74
77
79
81
84
87
90
93
96
99
101
104
107
110
112
114
115
116
118
120
123
125
128
131
133
136
138
141
143
145
148
151
154
157
159
162
164
166
169
172
173
175
178
181
184
187
190
192
195
198
201
204
207
210
212
214
216
218
220
222
224
227
230
233
Starting iteration 0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
3
5
6
8
10
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
52
53
54
55
57
59
61
63
66
68
71
74
76
78
81
84
87
90
93
96
98
101
104
107
109
112
113
115
117
119
122
124
127
130
132
135
137
140
142
144
147
150
153
156
158
161
163
165
168
171
173
175
178
180
183
186
189
191
194
197
200
203
206
209
211
213
215
217
219
221
223
226
229
232
Iteration 0 - Conflicts: 232 - Elite Fitness: None - Time Elapsed: 0.001611948013305664
Iteration 0 finished
Starting iteration 1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2

In [27]:
metrics_evaluator.metrics.best_iteration.avg_conflicts

227

In [29]:
avg_elasped_time = sum(iteration.time_elapsed for iteration in metrics_evaluator.metrics.iteration_history) / len(metrics_evaluator.metrics.iteration_history)
avg_elasped_time

0.0007137284278869629

In [30]:
avg_conflicts = sum([iteration.avg_conflicts for iteration in metrics_evaluator.metrics.iteration_history]) / len(metrics_evaluator.metrics.iteration_history)
avg_conflicts

231.101

In [28]:
metrics_evaluator.metrics.best_iteration.chromosome

[{'id': 1,
  'name': 'Apha',
  'schedule': [{'day': 'Segunda-Feira',
    'periods': [{'day': 'Segunda-Feira',
      'time': '8:00am - 9:00am',
      'subject': 'História',
      'teacher': 'Prof. Michael Brown',
      'room': 'Room 101'},
     {'day': 'Segunda-Feira',
      'time': '9:00am - 10:00am',
      'subject': 'Biologia',
      'teacher': 'Prof. Sarah Lee',
      'room': 'Room 101'},
     {'day': 'Segunda-Feira',
      'time': '10:00am - 11:00am',
      'subject': 'Física',
      'teacher': 'Prof. David Johnson',
      'room': 'Room 104'},
     {'day': 'Segunda-Feira',
      'time': '11:00am - 12:00pm',
      'subject': 'História',
      'teacher': 'Prof. Michael Brown',
      'room': 'Room 104'},
     {'day': 'Segunda-Feira',
      'time': '1:00pm - 2:00pm',
      'subject': 'História',
      'teacher': 'Prof. Michael Brown',
      'room': 'Room 102'}]},
   {'day': 'Terça-Feira',
    'periods': [{'day': 'Terça-Feira',
      'time': '8:00am - 9:00am',
      'subject': 'História